In [9]:
import requests
from xml.etree import ElementTree as ET
import time

# Configuration
SEARCH_URL = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi"
EMAIL = "your_email@example.com"  # Replace with your email
API_KEY = None     # Replace with your NCBI API key (optional)

def search_pubmed(query, max_results=20):
    """
    Search PubMed for a given query and return a list of PMIDs.
    """
    params = {
        "db": "pubmed",
        "term": query,
        "retmax": max_results,
        "retmode": "xml",
        "email": EMAIL,
    }
    if API_KEY:
        params["api_key"] = API_KEY

    response = requests.get(SEARCH_URL, params=params)
    if response.status_code != 200:
        print(f"Error fetching search results: {response.status_code}")
        return []

    root = ET.fromstring(response.text)
    pmids = [id_elem.text for id_elem in root.findall(".//Id")]
    return pmids

# Example Usage
if __name__ == "__main__":
    query = "machine learning in healthcare"
    pmids = search_pubmed(query, max_results=10)
    print(f"Found {len(pmids)} articles.")
    print(pmids)


Found 10 articles.
['39760233', '39759728', '39759471', '39759423', '39759343', '39759321', '39759269', '39758195', '39757989', '39757652']


In [30]:
FETCH_URL = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi"

from xml.etree import ElementTree as ET

def explore_article(article):
    """
    Print the full XML structure of a PubmedArticle object.
    """
    print(ET.tostring(article, encoding='unicode', method='xml'))



def fetch_pubmed_details(pmids):
    """
    Fetch detailed information for a list of PMIDs.
    """
    ids = ",".join(pmids)
    params = {
        "db": "pubmed",
        "id": ids,
        "retmode": "xml",
        "rettype": "xml",
        "email": EMAIL,
    }
    if API_KEY:
        params["api_key"] = API_KEY

    response = requests.get(FETCH_URL, params=params)
    if response.status_code != 200:
        print(f"Error fetching details: {response.status_code}")
        return []

    root = ET.fromstring(response.text)
    articles = []
    for article in root.findall(".//PubmedArticle"):
        title = article.find(".//ArticleTitle").text if article.find(".//ArticleTitle") is not None else "No Title"
        journal = article.find(".//Journal/Title").text if article.find(".//Journal/Title") is not None else "No Journal"
        pmid = article.find(".//PMID").text if article.find(".//PMID") is not None else "No PMID"
        articles.append({
            "title": title,
            "journal": journal,
            "pmid": pmid,
            "object": article
        })

        # Example usage
        # Assuming `article` is a <PubmedArticle> object
        explore_article(article)


    return articles

# Example Usage
if __name__ == "__main__":
    query = "machine learning in healthcare"
    pmids = search_pubmed(query, max_results=5)
    articles = fetch_pubmed_details(pmids)
    for article in articles:
        print(f"Title: {article['title']}")
        print(f"Journal: {article['journal']}")
        print(f"PMID: {article['pmid']}")
        print("-" * 50)


<PubmedArticle><MedlineCitation Status="In-Process" Owner="NLM"><PMID Version="1">39760233</PMID><DateRevised><Year>2025</Year><Month>01</Month><Day>06</Day></DateRevised><Article PubModel="Print"><Journal><ISSN IssnType="Electronic">2398-9238</ISSN><JournalIssue CitedMedium="Internet"><Volume>8</Volume><Issue>1</Issue><PubDate><Year>2025</Year><Month>Jan</Month></PubDate></JournalIssue><Title>Endocrinology, diabetes &amp; metabolism</Title><ISOAbbreviation>Endocrinol Diabetes Metab</ISOAbbreviation></Journal><ArticleTitle>Diagnosis Osteoporosis Risk: Using Machine Learning Algorithms Among Fasa Adults Cohort Study (FACS).</ArticleTitle><Pagination><StartPage>e70023</StartPage><MedlinePgn>e70023</MedlinePgn></Pagination><ELocationID EIdType="doi" ValidYN="Y">10.1002/edm2.70023</ELocationID><Abstract><AbstractText Label="INTRODUCTION" NlmCategory="BACKGROUND">In Iran, the assessment of osteoporosis through tools like dual-energy X-ray absorptiometry poses significant challenges due to t

In [31]:
obj = articles[0]['object']
obj


<Element 'PubmedArticle' at 0x13cb77650>

In [32]:
explore_article(obj)

<PubmedArticle><MedlineCitation Status="In-Process" Owner="NLM"><PMID Version="1">39760233</PMID><DateRevised><Year>2025</Year><Month>01</Month><Day>06</Day></DateRevised><Article PubModel="Print"><Journal><ISSN IssnType="Electronic">2398-9238</ISSN><JournalIssue CitedMedium="Internet"><Volume>8</Volume><Issue>1</Issue><PubDate><Year>2025</Year><Month>Jan</Month></PubDate></JournalIssue><Title>Endocrinology, diabetes &amp; metabolism</Title><ISOAbbreviation>Endocrinol Diabetes Metab</ISOAbbreviation></Journal><ArticleTitle>Diagnosis Osteoporosis Risk: Using Machine Learning Algorithms Among Fasa Adults Cohort Study (FACS).</ArticleTitle><Pagination><StartPage>e70023</StartPage><MedlinePgn>e70023</MedlinePgn></Pagination><ELocationID EIdType="doi" ValidYN="Y">10.1002/edm2.70023</ELocationID><Abstract><AbstractText Label="INTRODUCTION" NlmCategory="BACKGROUND">In Iran, the assessment of osteoporosis through tools like dual-energy X-ray absorptiometry poses significant challenges due to t

[]

In [11]:
def extract_doi(article):
    """
    Extract DOI from a PubMed article.
    """
    for elocation in article.findall(".//ELocationID"):
        if elocation.attrib.get("EIdType") == "doi":
            return elocation.text
    return None

# Example Usage
if __name__ == "__main__":
    query = "machine learning in healthcare"
    pmids = search_pubmed(query, max_results=5)
    articles = fetch_pubmed_details(pmids)
    
    for article in articles:
        print(f"Title: {article['title']}")
        print(f"Journal: {article['journal']}")
        print(f"PMID: {article['pmid']}")
        # Pass the XML element of the article to extract_doi
        doi = extract_doi(article)
        print(f"DOI: {doi if doi else 'N/A'}")
        print("-" * 50)


Title: Diagnosis Osteoporosis Risk: Using Machine Learning Algorithms Among Fasa Adults Cohort Study (FACS).
Journal: Endocrinology, diabetes & metabolism
PMID: 39760233


AttributeError: 'dict' object has no attribute 'findall'

In [6]:
queries = [{"section":"main", "query":"test"}]

result = await pubmed_client.query(queries)
result

{'main': [{'query': 'test',
   'pmid': '39761076',
   'title': 'How to Design an Inhaled Drug: CHF-6523.',
   'abstract': 'Inhalation provides an opportunity to target drugs to the lung, potentially improving efficacy and safety margins for the treatment of respiratory conditions. The discovery of inhaled medicines is a specialized endeavor and has different challenges to medicinal chemistry for oral drugs. The appearance of a successful campaign delivering an inhaled PI3Kδ inhibitor is therefore an opportunity to highlight how a team can address the challenges involved in the identification of such a compound. This Viewpoint seeks to highlight a great example of medicinal chemistry in the inhaled space.',
   'authors': ['Perry Matthew W D'],
   'journal': 'Journal of medicinal chemistry',
   'publication_date': '2025',
   'doi': '10.1021/acs.jmedchem.4c03130',
   'pubmed_url': 'https://pubmed.ncbi.nlm.nih.gov/39761076/'},
  {'query': 'test',
   'pmid': '39761063',
   'title': 'An Acet